In [ ]:
%pip install -U langchain-huggingface

In [ ]:
%pip install chroma-migrate

## 라이브러리 import

In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, DirectoryLoader
from collections import Counter, defaultdict
from langchain.docstore.document import Document
import os
import glob
import json
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI # 랭체인 llm용


## openai 연결

In [2]:

# .env 파일에서 환경 변수를 로드
load_dotenv("/data/ephemeral/home/upstage-ai-advanced-ir2-private/OPENAI_API_KEY.env")

# API 키 불러오기
api_key = os.getenv("OPENAI_API_KEY")

# API 키를 사용하여 설정
if api_key:
    os.environ["OPENAI_API_KEY"] = api_key
else:
    raise ValueError("API key not found. Please check your .env file.")

#print(f"OpenAI API Key: {api_key}")  # 필요시 확인용


## 데이터 불러오기 & 내용 확인

In [3]:
# JSONL 파일을 로드하기 위한 커스텀 로더
class JSONLLoader:
    def __init__(self, directory, file_pattern):
        self.directory = directory
        self.file_pattern = file_pattern

    def load(self):
        # 주어진 디렉토리에서 파일 패턴에 맞는 파일들 찾기
        file_paths = glob.glob(os.path.join(self.directory, self.file_pattern))
        documents = []
        # 각 파일에서 데이터를 읽고 JSON으로 변환하여 리스트에 추가
        for file_path in file_paths:
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    document = json.loads(line.strip())
                    documents.append(document)
        return documents

loader = JSONLLoader(
    '/data/ephemeral/home/upstage-ai-advanced-ir2-private/data/',
    'documents.jsonl'
)
documents = loader.load()
print('문서의 개수:', len(documents))


문서의 개수: 4272


In [4]:
print('1번 문서 :', documents[1])
print('-' * 20)
print('21번 문서 :', documents[21])
print('-' * 20)

1번 문서 : {'docid': '4a437e7f-16c1-4c62-96b9-f173d44f4339', 'src': 'ko_mmlu__conceptual_physics__test', 'content': '수소, 산소, 질소 가스의 혼합물에서 평균 속도가 가장 빠른 분자는 수소입니다. 수소 분자는 가장 가볍고 작은 원자로 구성되어 있기 때문에 다른 분자들보다 더 빠르게 움직입니다. 이러한 이유로 수소 분자는 주어진 온도에서 가장 빠른 평균 속도를 가지고 있습니다. 수소 분자는 화학 반응에서도 활발하게 참여하며, 수소 연료로도 널리 사용됩니다. 따라서 수소 분자는 주어진 온도에서 평균 속도가 가장 빠른 분자입니다.'}
--------------------
21번 문서 : {'docid': '7f3f4e9a-914b-4eae-8cf9-88f3879f6051', 'src': 'ko_mmlu__computer_security__test', 'content': '웹 프록시는 웹 브라우저와 웹 서버 간의 요청과 응답을 가로채고, 필요한 경우 수정하는 소프트웨어 조각입니다. 이는 사용자가 웹 브라우저를 통해 웹 서버에 접속할 때 중간에 위치하여 데이터를 중계하는 역할을 합니다. 웹 프록시는 사용자의 요청을 받아서 웹 서버로 전달하고, 웹 서버의 응답을 받아서 사용자에게 전달합니다. 이 과정에서 웹 프록시는 요청과 응답을 가로채어 필요한 경우 수정할 수 있습니다. 예를 들어, 웹 프록시는 광고를 제거하거나 웹 페이지의 콘텐츠를 압축하여 더 빠른 로딩을 제공할 수 있습니다. 또한, 웹 프록시는 보안을 강화하기 위해 사용자의 요청을 검사하고 악성 코드를 차단할 수도 있습니다. 웹 프록시는 네트워크 트래픽을 모니터링하고 분석하는 기능도 가지고 있어서, 네트워크 관리자가 트래픽 상황을 파악하고 최적화할 수 있습니다. 웹 프록시는 웹 서버와 웹 브라우저 사이에서 중개자 역할을 수행하여 웹 사용 경험을 개선하는데 도움을 줍니다.'}
--------------------


## 데이터 청크 분할

In [5]:
# 1. 문서 로드 및 청크 분할
# Document 객체로 변환 (page_content는 content 필드로 설정, chunkid는 존재할 때만 사용)
document_objects = [
    Document(
        page_content=doc["content"],
        metadata={
            "docid": doc["docid"],
            "src": doc["src"]  # chunkid가 없으면 None으로 설정
        }
    ) for doc in documents
]

# 텍스트 분할기 정의
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)

# Document 객체에서 content를 분할
split_documents = text_splitter.split_documents(document_objects)

# 분할된 텍스트의 개수를 출력
print('분할된 텍스트의 개수 :', len(split_documents))


분할된 텍스트의 개수 : 4921


In [6]:
split_documents[0]

Document(metadata={'docid': '42508ee0-c543-4338-878e-d98c6babee66', 'src': 'ko_mmlu__nutrition__test'}, page_content='건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.')

In [7]:
# 2. 문서 청크별로 docid를 기반으로 추적
# docid를 기반으로 각 청크를 추적
docid_list = [doc.metadata['docid'] for doc in split_documents]

# 문서별 청크 개수 계산
docid_counts = Counter(docid_list)

# 2개 이상의 청크로 분할된 문서만 필터링
filtered_counts = {key: value for key, value in docid_counts.items() if value >= 2}

# 2개 이상 분할된 문서 출력
print('2개 이상으로 분할된 문서 :', filtered_counts)

# 전체 분할된 텍스트의 개수 출력
print('분할된 텍스트의 개수 :', len(split_documents))

2개 이상으로 분할된 문서 : {'d3c68be5-9cb1-4d6e-ba18-5f81cf89affb': 2, '74f22819-1a8e-4646-8a9d-13323de8cdb8': 2, '80feb7f2-1b24-4a9d-87a3-4976e9304e74': 2, '418e1e6c-c2de-4ffb-aded-0992fe833776': 2, '7245d870-2ead-460b-97b2-aa6208c4ba29': 2, '7f3f4e9a-914b-4eae-8cf9-88f3879f6051': 2, 'e6c6ebd0-cb3a-4c52-955c-37723cd13864': 2, 'a65fb25c-f9c0-4822-b824-71141f2b6a09': 2, '814dd12b-62a4-4a79-99d2-82fa39ba76e3': 2, '5c6a539a-b7c9-43ff-ac7d-c92501af5a5b': 2, 'dd5fff95-420f-46fa-a624-5e74f73e56a7': 2, '67d746ab-9fe8-4945-b986-4fb4bf0cbd10': 2, '31b1e0a2-699e-49e6-97a1-6be60a5ee1a0': 2, 'df7dd7f7-7ed9-4e8e-bf7d-546c0c7fee92': 2, 'eda417a2-9251-4fa4-bd7b-dcc7129c9620': 2, '548598c3-eec1-4c0c-8c0f-f6c3ca65b010': 2, '949507ee-6275-4b21-9ebc-eec46a7e89d2': 2, '209121c3-69df-414a-a1b9-c74a1b14384f': 2, '233a0770-8633-46ab-b1e2-ac351df1b50e': 2, '35c5dcc7-4720-4318-901e-770105ae63fd': 2, '17e6e494-dbc2-4d6f-92b7-c160a0b47a8b': 2, '00c4ac0d-e778-4a02-b725-f25b97109d75': 2, 'a4370eb4-6677-4bb8-a1e0-ee3d5410790

## ----- vector store에 저장  ** 저장해둔 것 있으면 아래로 내려가서 불러오기만 하기 ** ----

In [ ]:
from chromadb.config import Settings
# 데이터가 저장될 디렉토리 설정
persist_directory = '/data/ephemeral/home/upstage-ai-advanced-ir2-private/data_langchain2/chroma_data_test2'  # 원하는 경로로 변경

embedding_model = OpenAIEmbeddings()

# Chroma 데이터베이스에 텍스트를 저장 (이미 분할된 Document 객체 리스트)
vectordb = Chroma.from_documents(
    documents=split_documents,  # 이미 분할된 Document 객체 리스트 (청크 데이터)
    embedding=embedding_model,  # SentenceTransformer 기반 임베딩 모델 사용
    persist_directory=persist_directory,  # 데이터가 저장될 경로 지정
    collection_name="my_collection",
)




print('Chroma 벡터 DB 생성 완료')

## --------------------- 저장한 DB 불러오기 --------------------------

In [ ]:
# %pip install langchain_chroma

In [13]:
from langchain_chroma import Chroma
# 저장된 인덱스가 있을 때
# 데이터가 저장될 디렉토리 설정
persist_directory = '/data/ephemeral/home/upstage-ai-advanced-ir2-private/data_langchain2/chroma_data_test2'  # 원하는 경로로 변경

# SentenceTransformerEmbeddings를 사용하여 임베딩 생성
# embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
embedding_model = OpenAIEmbeddings()

# 불러올때?
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_model,
    collection_name="my_collection"  # 기존 컬렉션 ID
)

### -------------------------------------------------------------------------------------

## Make a retriever & Test

In [14]:
retriever = vectordb.as_retriever() # 벡터 검색 방법은 괄호 안에 입력하면 됨

In [15]:
vectordb.similarity_search('복숭아 키우는 노하우')

[Document(metadata={'docid': 'b5f3daf1-1d09-4f1e-96d0-7f12116ab590', 'src': 'ko_ai2_arc__ARC_Challenge__test'}, page_content='그러나 사구에서 더 멀리 떨어진 곳에는 초원과 숲이 형성됩니다. 이는 빙하가 녹으면서 물이 더 멀리 퍼져나가기 때문입니다. 빙하가 녹으면서 초원과 숲이 형성될 수 있게 된 것입니다. 이러한 과정은 1차 천이라고 불립니다.'),
 Document(metadata={'docid': '393af029-ba90-4eee-9927-7d20b42c2a01', 'src': 'ko_ai2_arc__ARC_Challenge__train'}, page_content='빙하의 건설적 힘에 따른 결과로는 빙하가 녹아 퇴적된 암석 더미가 형성됩니다. 빙하는 수천 년 동안 땅을 강하게 압축하고, 얼음과 함께 암석과 흙을 이동시킵니다. 이러한 작용으로 인해 빙하가 지나간 지역에는 암석과 퇴적물이 쌓이게 됩니다. 이 암석 더미는 빙하의 힘에 의해 형성되었으며, 빙하가 녹아 퇴적된 결과물입니다. 이러한 지형은 주로 산악 지역이나 빙하가 존재했던 지역에서 발견됩니다. 빙하의 건설적 힘은 자연의 힘 중 하나로, 우리가 살고 있는 지구의 지형 형성에 큰 영향을 미치는 중요한 요소입니다.'),
 Document(metadata={'docid': 'bec3008a-9c61-4e59-b6a1-52b34666ca10', 'src': 'ko_ai2_arc__ARC_Challenge__validation'}, page_content='초콜릿은 더운 날 햇볕 아래에 놓여있을 때, 녹을 수 있습니다. 이 과정에서 초콜릿의 모양이 변하게 됩니다. 녹은 초콜릿은 원래의 고체 모양에서 액체로 변하게 되며, 이는 초콜릿의 특성 중 하나입니다. 따라서, 더운 날 햇볕 아래에 놓인 초콜릿은 모양이 변하게 됩니다.'),
 Document(metadata={'docid': '6a93bf30-f16d-4

In [16]:
docs = retriever.get_relevant_documents("세균의 순기능")
print('유사 문서 개수 :', len(docs))
print('--' * 20)
print('첫번째 유사 문서 :', docs[0])
print('--' * 20)
print('각 유사 문서의 문서 출처 :')
for doc in docs:
    print(doc.metadata["docid"])

유사 문서 개수 : 4
----------------------------------------
첫번째 유사 문서 : page_content='세균은 무성 생식을 하는 유기체입니다. 이는 세균이 단일 부모로부터 새롭게 생산되는 것을 의미합니다. 세균은 유전자를 상속받는데, 이는 단일 부모와 같은 특성을 가지게 됩니다. 세균은 유전자의 조합을 통해 다양한 특성을 가질 수 있으며, 이는 세균의 다양성과 적응력을 높여줍니다. 세균은 빠른 성장과 번식 능력으로 인해 환경에서 매우 중요한 역할을 합니다. 세균은 다양한 환경에서 생존할 수 있으며, 이는 세균의 유전적 다양성과 적응력에 기인합니다. 세균은 우리 주변에 많이 존재하며, 우리의 건강에도 영향을 미칩니다. 세균의 특성을 이해하고 관리하는 것은 우리의 건강과 생활에 매우 중요합니다.' metadata={'docid': '5c5a0200-462e-4dd6-981f-b0bf7026ec5e', 'src': 'ko_ai2_arc__ARC_Challenge__test'}
----------------------------------------
각 유사 문서의 문서 출처 :
5c5a0200-462e-4dd6-981f-b0bf7026ec5e
e0458408-2b78-47c0-b28d-55cc5cab214c
1655c90b-29c7-47ef-a092-01f2550db3aa
85d28a10-9380-4afe-afef-b34449ef86bf


### 결과를 k개 반환

In [17]:
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [18]:
docs = retriever.get_relevant_documents("세균의 순기능")

for doc in docs:
    print(doc.metadata["docid"], doc.page_content)

5c5a0200-462e-4dd6-981f-b0bf7026ec5e 세균은 무성 생식을 하는 유기체입니다. 이는 세균이 단일 부모로부터 새롭게 생산되는 것을 의미합니다. 세균은 유전자를 상속받는데, 이는 단일 부모와 같은 특성을 가지게 됩니다. 세균은 유전자의 조합을 통해 다양한 특성을 가질 수 있으며, 이는 세균의 다양성과 적응력을 높여줍니다. 세균은 빠른 성장과 번식 능력으로 인해 환경에서 매우 중요한 역할을 합니다. 세균은 다양한 환경에서 생존할 수 있으며, 이는 세균의 유전적 다양성과 적응력에 기인합니다. 세균은 우리 주변에 많이 존재하며, 우리의 건강에도 영향을 미칩니다. 세균의 특성을 이해하고 관리하는 것은 우리의 건강과 생활에 매우 중요합니다.
e0458408-2b78-47c0-b28d-55cc5cab214c 세균은 진화가 가장 빠르게 발생할 수 있는 생물체 중 하나입니다. 세균은 단세포 생물체로서, 매우 짧은 세대간 시간을 가지고 번식합니다. 이러한 빠른 번식 속도는 세균이 짧은 세대간 동안 다양한 유전자 변이를 쌓을 수 있게 합니다. 이러한 변이는 세균의 생존과 번성에 중요한 역할을 합니다. 또한, 세균은 환경 변화에 빠르게 적응할 수 있는 능력을 가지고 있습니다. 이는 세균이 다양한 환경에서 살아남고 번식할 수 있게 해주는데 도움을 줍니다. 따라서, 세균은 진화가 가장 빠르게 발생할 수 있는 생물체 중 하나로 알려져 있습니다.
1655c90b-29c7-47ef-a092-01f2550db3aa 세균은 다양한 방식으로 인체에 도움을 줄 수 있습니다. 대표적으로, 세균은 음식을 분해하는 데 도움을 줍니다. 세균은 소화 과정에서 음식물을 분해하여 영양소를 생성하고 흡수를 도와줍니다. 이를 통해 우리 몸은 필요한 영양소를 효과적으로 흡수할 수 있습니다. 따라서, 세균은 우리 인체와 환경에 많은 도움을 주는 중요한 존재입니다.
85d28a10-9380-4afe-afef-b34449ef86bf 생태계에서 죽은 식물과 동물을 재활용하는 데 책임이 가장 큰 것

## Make a chain


```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

In [21]:
# LLM 설정 및 질의응답 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)


In [22]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# LLM 및 리트리버 설정
llm = ChatOpenAI()
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

# 시스템 프롬프트 설정
system_prompt = (
    "주어진 문맥을 사용해 질문에 답변하세요. "
    "모르면 모른다고 답변하세요. "
    "최대 세 문장으로 간결하게 답변을 생성하세요. "
    "문맥: {context}"
)

# ChatPromptTemplate을 사용하여 프롬프트 구성
retrieval_qa_chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# 프롬프트와 문서 결합 체인 생성
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 질의응답 수행
result = rag_chain.invoke({"input": "동물의 털 색깔"})

# 답변 출력
print(result['answer'])

# 문서 출처 및 내용 출력
print("\n참고한 문서들:")
for doc in result["context"]:
    print(f"DocID: {doc.metadata['docid']}")
    print(f"Content: {doc.page_content}\n")


동물의 털 색깔은 생존에 도움을 주는 중요한 역할을 합니다. 동물들은 환경에 녹아들 수 있도록 색상을 조절하여 먹이나 적의 시선을 피하고, 사냥을 하거나 사냥을 피할 수 있습니다. 또한, 털의 색깔이 변화하는 것은 계절이나 환경 변화에 대한 동물들의 적응 메커니즘이라고 볼 수 있습니다.

참고한 문서들:
DocID: 5973b08a-ef7d-431c-ac58-d31d4a57d63c
Content: 동물들은 자신의 환경에 녹아들 수 있도록 색상과 패턴을 적용합니다. 이러한 특성을 적용한 이유는 숨어 있기 위해서입니다. 동물들은 자신의 생존을 위해 환경과 조화를 이루기 위해 색상과 패턴을 활용합니다. 이를 통해 동물들은 먹이나 적의 시선을 피하고, 사냥을 하거나 사냥을 피하는 데에 도움을 받을 수 있습니다. 예를 들어, 사자는 노란 갈색의 털을 가지고 있어 사방에서 녹아들 수 있습니다. 이렇게 사자는 사냥감에게 자신의 존재를 드러내지 않고 접근할 수 있습니다. 또한, 동물들은 자신의 환경에 맞는 색상과 패턴을 선택함으로써 서식지에서의 경쟁에서 우위를 점할 수 있습니다. 예를 들어, 암컷 호랑이는 녹색의 무늬를 가지고 있어 정글에서 녹아들 수 있습니다. 이렇게 호랑이는 경쟁 상대들에게 자신의 존재를

DocID: fd84a7cd-1309-45e4-8811-1a89bd432e7c
Content: 색상은 동물이 짝을 유혹하는 데 종종 도움이 됩니다. 많은 동물들은 색상을 사용하여 상대방의 관심을 끌고, 짝짓기를 유도합니다. 예를 들어, 새들은 화려한 깃털을 가지고 있어서 파트너를 유혹하고 매력적으로 보이게 합니다. 또한, 어떤 동물들은 특정한 색상을 가진 꽃이나 과일을 찾아가서 먹음으로써 짝을 유혹합니다. 이러한 적응은 동물들이 번식을 위해 상대방을 찾는 데 도움이 되며, 종의 생존을 보장하는 역할을 합니다.

DocID: cd00611c-aa67-42be-9981-5e4733e1fadd
Content: 특정 동물은 색을 빠르게 변화시킬 수 있는 분화된 세포를 가지고 

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# DB에서 원본 문서를 가져오는 함수
def get_original_document_by_docid(docid):
    # documents 리스트에서 해당 docid를 찾아 원본 content를 반환
    for doc in documents:
        if doc['docid'] == docid:
            return doc['content']
    return "원본 문서를 찾을 수 없습니다."

# LLM 및 리트리버 설정
llm = ChatOpenAI()
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

# 시스템 프롬프트 설정
system_prompt = (
    "주어진 문맥을 사용해 질문에 답변하세요. "
    "모르면 모른다고 답변하세요. "
    "최대 세 문장으로 간결하게 답변을 생성하세요. "
    "문맥: {context}"
)

# ChatPromptTemplate을 사용하여 프롬프트 구성
retrieval_qa_chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# 프롬프트와 문서 결합 체인 생성
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 질의응답 수행
result = rag_chain.invoke({"input": "동물의 털 색깔"})

# 답변 출력
print(result['answer'])

# 문서 출처 및 DB에서 원본 문서 가져오기
print("\n참고한 문서들:")
for doc in result["context"]:
    docid = doc.metadata['docid']  # 청크된 문서의 docid 가져오기
    original_doc = get_original_document_by_docid(docid)  # 원본 문서에서 해당 docid로 검색하여 가져오기
    print(f"DocID: {docid}")
    print(f"Original Content: {original_doc}\n")  # 원본 문서 내용 출력


In [ ]:
import logging

# # 디버깅 활성화 (LLM 쿼리와 체인의 동작을 추적하기 위함)
# logging.basicConfig(level=logging.DEBUG)
# ------------------------------------------------------------------------------------------------
# 전체 로깅 수준을 INFO로 설정 (필요에 따라 WARNING으로 변경 가능)
# logging.basicConfig(level=logging.INFO)

# # OpenAI와 httpx 관련 디버깅을 명시적으로 비활성화
# logging.getLogger("httpx").setLevel(logging.WARNING)
# logging.getLogger("httpcore").setLevel(logging.WARNING)
# logging.getLogger("openai").setLevel(logging.WARNING)

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# DB에서 원본 문서를 가져오는 함수
def get_original_document_by_docid(docid):
    # documents 리스트에서 해당 docid를 찾아 원본 content를 반환
    for doc in documents:
        if doc['docid'] == docid:
            return doc['content']
    return "원본 문서를 찾을 수 없습니다."

# LLM 및 리트리버 설정
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

# 시스템 프롬프트 설정 (쿼리 확장을 유도하는 내용 추가)
system_prompt = (
    "주어진 문맥을 사용해 질문에 답변하고, 답변을 더 풍부하게 하기 위해 관련된 추가 질문을 5개 생성하세요. "
    "질문은 명사구로 간결하게 생성하세요"
    "질문에 대한 키워드도 생성하세요."
    "모르면 모른다고 답변하세요. "
    "최대 세 문장으로 간결하게 답변을 생성하세요. "
    "문맥: {context}"
)

# ChatPromptTemplate을 사용하여 프롬프트 구성
retrieval_qa_chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# 프롬프트와 문서 결합 체인 생성
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 질의응답 수행
result = rag_chain.invoke({"input": "복숭아 키우는 노하우 좀"})

# 답변 출력
print(result['answer'])

# 문서 출처 및 DB에서 원본 문서 가져오기
print("\n참고한 문서들:")
for doc in result["context"]:
    docid = doc.metadata['docid']  # 청크된 문서의 docid 가져오기
    original_doc = get_original_document_by_docid(docid)  # 원본 문서에서 해당 docid로 검색하여 가져오기
    print(f"DocID: {docid}")
    print(f"Original Content: {original_doc}\n")  # 원본 문서 내용 출력


## Query

In [ ]:
# qa_chain은 LangChain에서 생성된 질의응답(Question Answering) 체인

input_text = "세균의 순기능"
chatbot_response = qa_chain(input_text)
print(chatbot_response)

In [62]:
# **챗봇의 응답(chatbot_response)**을 출력하고, 응답에 사용된 문서의 출처를 함께 보여주는 역할
def get_chatbot_response(chatbot_response):
    print(chatbot_response['result'].strip())
    print('\n문서 출처:')
    for source in chatbot_response["source_documents"]:
        print(source.metadata['docid'], doc.page_content)

In [ ]:
input_text = "세균의 순기능"
chatbot_response = qa_chain(input_text)
get_chatbot_response(chatbot_response)

In [ ]:
# print(chatbot_response)
chatbot_response

In [ ]:
input_text = "전세자금이 부족한 사람을 위한 정책 이름이 뭐야?"
llm_response = qa_chain(input_text)
get_chatbot_response(llm_response)

In [ ]:
query = "동물의 털 색깔"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

In [ ]:
query = "희망두배 청년통장의 지원 조건은?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

In [ ]:
query = "희망두배 청년통장의 마감 기한은?"
llm_response = qa_chain(query)
get_chatbot_response(llm_response)

In [ ]:
!pip install gradio

To create a public link, set `share=True` in `launch()`.  
Running on https://localhost:7860/  

라는 식의 문구가 나오면 위의 localhost로 시작하는 주소를 클릭하세요.

In [ ]:
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="청년정책챗봇") # 청년정책챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창의 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      result = qa_chain(message)
      bot_message = result['result']
      bot_message += ' # sources :'

      # 답변의 출처를 표기
      for i, doc in enumerate(result['source_documents']):
          bot_message += '[' + str(i+1) + '] ' + doc.metadata['source'] + ' '

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True)